# Imports

In [3]:
from selenium import webdriver
import pandas as pd

# Definition du parser

In [153]:
def parse_s(url):
    headers_table = []
    results_table=[]
    results_table_final = []
    driver = webdriver.Chrome()
    driver.get(url)
    
    rows = driver.find_elements_by_css_selector("#yw1 > table")
    
    #Recuperation des results:

    for row in rows:
        temp = []
        columns = row.find_elements_by_css_selector("td")
        for column in columns:
            temp.append(column.text)
        results_table.append(temp)
    results_table_final=[results_table[0][i*9+3:9*(i+1)] for i in range(len(results_table[0])//9)]
    driver.close()
    
    return results_table_final

# Recuperation des urls

Il suffit de changer l'année à la fin de l'url:

Premier League : 'https://www.transfermarkt.fr/premier-league/startseite/wettbewerb/GB1/plus/?saison_id=2018'

La Liga        : 'https://www.transfermarkt.fr/laliga/startseite/wettbewerb/ES1/plus/?saison_id=2018'

Serie A        : 'https://www.transfermarkt.fr/serie-a/startseite/wettbewerb/IT1/plus/?saison_id=2018'

Bundesliga     : 'https://www.transfermarkt.fr/1-bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2018'

Ligue 1        : 'https://www.transfermarkt.fr/ligue-1/startseite/wettbewerb/FR1/plus/?saison_id=2018'

On ouvre la page d'index et on récupère tous les url dans la table des clubs.

In [296]:
url='https://www.transfermarkt.fr/ligue-1/startseite/wettbewerb/FR1/plus/?saison_id=2017'
driver = webdriver.Chrome()
driver.get(url)

url_table =[]
for a in driver.find_elements_by_xpath('//*[@id="yw1"]/table/tbody//a'):
    url_table.append(a.get_attribute('href'))
driver.close()
url_table

['https://www.transfermarkt.fr/fc-paris-saint-germain/startseite/verein/583/saison_id/2017',
 'https://www.transfermarkt.fr/fc-paris-saint-germain/startseite/verein/583/saison_id/2017',
 'https://www.transfermarkt.fr/ligue-1/startseite/wettbewerb/FR1/plus/?saison_id=2017#',
 'https://www.transfermarkt.fr/fc-paris-saint-germain/startseite/verein/583/saison_id/2017',
 'https://www.transfermarkt.fr/fc-paris-saint-germain/kader/verein/583/saison_id/2017',
 'https://www.transfermarkt.fr/fc-paris-saint-germain/kader/verein/583/saison_id/2017',
 'https://www.transfermarkt.fr/fc-paris-saint-germain/kader/verein/583/saison_id/2017',
 'https://www.transfermarkt.fr/as-monaco/startseite/verein/162/saison_id/2017',
 'https://www.transfermarkt.fr/as-monaco/startseite/verein/162/saison_id/2017',
 'https://www.transfermarkt.fr/ligue-1/startseite/wettbewerb/FR1/plus/?saison_id=2017#',
 'https://www.transfermarkt.fr/as-monaco/startseite/verein/162/saison_id/2017',
 'https://www.transfermarkt.fr/as-monac

On garde simplement les url vers les pages des clubs

In [297]:
url_final=[]
url_final.append(url_table[0])
url_final.append(url_table[8])
for i in range(18):
    url_final.append(url_table[14+6*i])

url_final                  

['https://www.transfermarkt.fr/fc-paris-saint-germain/startseite/verein/583/saison_id/2017',
 'https://www.transfermarkt.fr/as-monaco/startseite/verein/162/saison_id/2017',
 'https://www.transfermarkt.fr/olympique-marseille/startseite/verein/244/saison_id/2017',
 'https://www.transfermarkt.fr/ogc-nizza/startseite/verein/417/saison_id/2017',
 'https://www.transfermarkt.fr/olympique-lyon/startseite/verein/1041/saison_id/2017',
 'https://www.transfermarkt.fr/as-saint-etienne/startseite/verein/618/saison_id/2017',
 'https://www.transfermarkt.fr/fc-girondins-bordeaux/startseite/verein/40/saison_id/2017',
 'https://www.transfermarkt.fr/fc-toulouse/startseite/verein/415/saison_id/2017',
 'https://www.transfermarkt.fr/fc-stade-rennes/startseite/verein/273/saison_id/2017',
 'https://www.transfermarkt.fr/fc-nantes/startseite/verein/995/saison_id/2017',
 'https://www.transfermarkt.fr/ea-guingamp/startseite/verein/855/saison_id/2017',
 'https://www.transfermarkt.fr/sco-angers/startseite/verein/142

# Recuperation des donnees

In [304]:
headers_table = ['Player', 'Position', 'Né/âge','', 'Nat.', 'Market_Value']

In [298]:
results_table_final = []
for i in range(len(url_final)):
    results_table= parse_s(url_final[i])
    for player in results_table:
        results_table_final.append(player)
    

In [4]:
df = pd.DataFrame(results_table_final, columns=headers_table)
df

NameError: name 'results_table_final' is not defined

# Nettoyage des donnees

In [ ]:
On réassigne les colonnes.

In [306]:
new = df['Né/âge'].str.split("(", n = 1, expand = True) 

df['Naissance']= new[0] 
df = df.drop(columns=['Nat.', 'Né/âge', '' ])
df

,Player,Position,Market_Value,Naissance
0,Kevin Trapp,Gardien de but,"10,00 mio. €",8 juil. 1990
1,Alphonse Areola,Gardien de but,"10,00 mio. €",27 févr. 1993
2,Rémy Descamps,Gardien de but,150 K €,25 juin 1996
3,Sébastien Cibois,Gardien de but,,2 mars 1998
4,Marquinhos,Défenseur central,"35,00 mio. €",14 mai 1994
...,...,...,...,...
678,Idriss Saadi,Avant centre,"2,00 mio. €",8 févr. 1992
679,Stéphane Bahoken,Avant centre,800 K €,28 mai 1992
680,Nuno Da Costa,Avant centre,600 K €,10 févr. 1991
681,Jérémy Blayac,Avant centre,250 K €,13 juin 1983


In [307]:
df = df[['Player', 'Position', 'Naissance', 'Market_Value']]
df

,Player,Position,Naissance,Market_Value
0,Kevin Trapp,Gardien de but,8 juil. 1990,"10,00 mio. €"
1,Alphonse Areola,Gardien de but,27 févr. 1993,"10,00 mio. €"
2,Rémy Descamps,Gardien de but,25 juin 1996,150 K €
3,Sébastien Cibois,Gardien de but,2 mars 1998,
4,Marquinhos,Défenseur central,14 mai 1994,"35,00 mio. €"
...,...,...,...,...
678,Idriss Saadi,Avant centre,8 févr. 1992,"2,00 mio. €"
679,Stéphane Bahoken,Avant centre,28 mai 1992,800 K €
680,Nuno Da Costa,Avant centre,10 févr. 1991,600 K €
681,Jérémy Blayac,Avant centre,13 juin 1983,250 K €


On supprime les joueurs sans valeur marchande

In [26]:
df = d.loc[d['Market Value']!=' ']
df

,Unnamed: 0,Player,Position,Naissance,Market Value
0,0,Kevin Trapp,Gardien de but,8 juil. 1990,"10,00 mio. €"
1,1,Alphonse Areola,Gardien de but,27 févr. 1993,"10,00 mio. €"
2,2,Rémy Descamps,Gardien de but,25 juin 1996,150 K €
4,4,Marquinhos,Défenseur central,14 mai 1994,"35,00 mio. €"
5,5,Thiago Silva,Défenseur central,22 sept. 1984,"12,00 mio. €"
...,...,...,...,...,...
677,677,Martin Terrier,Ailier gauche,4 mars 1997,400 K €
678,678,Idriss Saadi,Avant centre,8 févr. 1992,"2,00 mio. €"
679,679,Stéphane Bahoken,Avant centre,28 mai 1992,800 K €
680,680,Nuno Da Costa,Avant centre,10 févr. 1991,600 K €


In [302]:
df.to_csv('Transfermarkt\L1_Market_2017-2018.csv')